In [ ]:
# STEP 1: Install packages
!pip install google-generativeai tqdm pandas

In [ ]:

# STEP 2: Imports
import os
import google.generativeai as genai
import pandas as pd
from tqdm import tqdm
from google.colab import drive
import re

In [ ]:
# STEP 3: Authenticate Gemini (replace the api_key)
genai.configure(api_key="")

In [ ]:
# STEP 4: Load Gemini 1.5 Pro or 2.5 Pro if available
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")
# model = genai.GenerativeModel(model_name="gemini-2.5-pro")

In [ ]:
# STEP 5: Mount Google Drive
drive.mount('/content/drive', force_remount=True)
input_folder = "/content/drive/MyDrive/research/practice_exams"
output_folder = "/content/drive/MyDrive/research/gen_responses"
os.makedirs(output_folder, exist_ok=True)

In [ ]:
# STEP 6: Exam files (FIXED commas)
exam_files = [
    # "CIPT Practice Exam.csv",
    # "CIPPUS Practice Exam.csv",
    # "AIGP Practice Exam.csv",
    "CIPM Practice Exam.csv",
]

In [ ]:
# STEP 7: Prompt builder
def build_prompt(row, answer_columns):
    lines = []

    if 'scenario' in row and pd.notna(row['scenario']):
        lines.append(f"Context:\n{row['scenario']}\n")

    lines.append(f"Question:\n{row['question']}\n")
    lines.append("Choices:")
    for i, col in enumerate(answer_columns):
        lines.append(f"{chr(65 + i)}. {row[col]}")

    lines.append(
        "\nYou are a certified U.S. privacy professional taking a high-stakes multiple-choice exam (such as the AIGP, CIPP/US, or CIPT).\n"
        "Read the question and choices carefully. Use your knowledge of U.S. privacy laws (e.g., GDPR, CCPA, HIPAA, etc.), data governance best practices, and legal reasoning.\n\n"
        "Eliminate clearly incorrect choices if possible. Choose the BEST answer, even if more than one seems partially correct.\n\n"
        "Respond only in the following exact format:\n"
        "Final Answer: <A/B/C/D>\n"
        "Explanation: <A concise justification, under 150 words. Reference relevant laws or best practices.>\n\n"
        "Do not explain all four choices—just support your final choice."
    )
    return "\n".join(lines)

In [ ]:
# STEP 8: Gemini Query with error handling
def query_model_gemini(prompt):
    try:
        response = model.generate_content(prompt)
        return response.text if response else ""
    except Exception as e:
        print(f"⚠️ Gemini query failed: {e}")
        return f"ERROR: {e}"

In [ ]:
# STEP 9: Extract answer letter
def extract_answer_letter(response, answer_choices=None):
    if not isinstance(response, str):
        return ""

    match = re.search(r"final answer\s*[:\-]?\s*([A-D])\b", response, re.IGNORECASE)
    if match:
        return match.group(1).upper()

    lines = response.strip().splitlines()
    if not lines:
        return ""

    for line in lines:
        if re.fullmatch(r"[A-Da-d]", line.strip()):
            return line.strip().upper()

    match = re.search(r"\b(correct|best|answer)\s*(is|:)?\s*([A-D])\b", response, re.IGNORECASE)
    if match:
        return match.group(3).upper()

    match = re.match(r"^\s*([A-D])[\.:]?\s", lines[0])
    if match:
        return match.group(1).upper()

    return ""

In [ ]:

# STEP 10: Evaluate response
def evaluate_response(model_letter, correct_letter, choices, response):
    model_letter = model_letter.strip().upper()
    correct_letter = correct_letter.strip().upper()

    if model_letter == correct_letter:
        return 1

    correct_text = choices.get(correct_letter, "").strip().lower()
    if correct_text and correct_text in response.lower():
        return 1

    return 0

In [ ]:
# STEP 11: Main loop
for exam_file in exam_files:
    print(f"📄 Processing {exam_file} with Gemini 1.5 Pro")

    file_path = os.path.join(input_folder, exam_file)
    df = pd.read_csv(file_path)

    # Check required columns
    answer_cols = sorted([col for col in df.columns if col.lower().startswith("answer")])
    required_cols = ['question', 'correct answer'] + answer_cols
    missing_cols = [col for col in required_cols if col not in df.columns]

    if missing_cols:
        print(f"⚠️ Skipping {exam_file} - missing columns: {missing_cols}")
        continue

    df = df.dropna(subset=['question'])

    results = []
    for _, row in tqdm(df.iterrows(), total=len(df), desc="💬 Asking Gemini"):
        prompt = build_prompt(row, answer_cols)
        response = query_model_gemini(prompt)
        answer_choices = {chr(65 + i): row[col] for i, col in enumerate(answer_cols)}
        model_letter = extract_answer_letter(response, answer_choices)
        score = evaluate_response(model_letter, row['correct answer'], answer_choices, response)

        result = {
            "question": row['question'],
            "scenario": row.get('scenario', ""),
            "correct answer": row['correct answer'],
            "model answer": model_letter,
            "score": score,
            "response": response,
            **{col: row[col] for col in answer_cols}
        }
        results.append(result)

    df_out = pd.DataFrame(results)

    # Add total score
    total_score = df_out['score'].sum()
    total_questions = len(df_out)
    percentage = (total_score / total_questions) * 100 if total_questions > 0 else 0

    total_row = {col: "" for col in df_out.columns}
    total_row['question'] = "TOTAL SCORE"
    total_row['score'] = f"{total_score}/{total_questions} ({percentage:.1f}%)"
    df_out = pd.concat([df_out, pd.DataFrame([total_row])], ignore_index=True)

    output_file = f"{exam_file.split('.')[0]}-gemini-1.5-pro-output.csv"
    df_out.to_csv(os.path.join(output_folder, output_file), index=False)
    print(f"✅ Saved to {output_file}")
    print(f"🎯 Score: {total_score}/{total_questions} ({percentage:.1f}%)")
